In [10]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import ast


import sys
import os
from pathlib import Path
import pymc3

AttributeError: partially initialized module 'theano' has no attribute 'compile' (most likely due to a circular import)

In [8]:
lib_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'lib'))
print("Lib path:", lib_path)

# Append the absolute path to sys.path
sys.path.append(lib_path)

# Now, try importing onedmodel
from onedmodel import *

Lib path: /Users/guime/projects/SLTforSGD/code/lib
